# ***CS:GO Round Winner Classification***

---

To develop a classification which can predict who wins individual snapshots of rounds.


#**Context**


*   CS:GO is a tactical shooter, where 2 teams (CT and Terrorist) play for best of 30 rounds.
*   There are 5 players
on each team (10 in total) and the first team to reach 16 rounds wins the game.
* At the start, one team plays as CT and the other as Terrorist. After 15 rounds
played, the teams swap sides.
* You win a round as a Terrorist by either planting the bomb and making sure it
explodes, or by eliminating the other team.
*  You win a round as CT by either
eliminating the other team, or by disarming the bomb, should it have been
planted.



#**Importing libraries and reading data**

In [ ]:
import pandas as pd
import numpy as np

Reading data:

In [ ]:
data = pd.read_csv('/content/csgo_round_snapshots.csv.zip')

In [ ]:
data

#**Data Exploration**

In [ ]:
#checking null values
data.isnull().any()

In [ ]:
data.isnull().sum().sum()

adding t_win col


In [ ]:
data['t_win'] = data.round_winner.astype("category").cat.codes

In [ ]:
data.describe()

In [ ]:
data.shape

#**Data Preprocessing**


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

Finding correlations and selecting top 25 columns

In [ ]:
correlations = data.corr()
print(correlations['t_win'].apply(abs).sort_values(ascending = False).iloc[:25])

In [ ]:
selected_columns = []
for col in data:
  try:
    if abs(correlations[col]['t_win']) > 0.15:
      selected_columns.append(col)
  except KeyError:
      pass
data_selected = data[selected_columns]
print(data_selected)

In [ ]:
data_selected


#**Data Visualization**

Plotting HeatMap

In [ ]:
plt.figure(figsize = (20,20))
sns.heatmap(data_selected.corr().sort_values(by = 't_win'), annot = True, cmap = "YlGnBu")

Plotting a Histogram

In [ ]:
data_selected.hist(figsize = (16,16))

Splitting data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X, y = data_selected.drop(['t_win'], axis = 1), data_selected['t_win']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

#**1. KNN Classification Algorithm**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

In [ ]:
knn.score(X_train_scaled, y_train)

In [ ]:
knn.score(X_test_scaled, y_test)

**ROC Curve**

In [ ]:
from sklearn.metrics import RocCurveDisplay

In [ ]:
RocCurveDisplay.from_estimator(knn, X_test_scaled, y_test)
plt.show()

In [ ]:
RocCurveDisplay.from_estimator(knn, X_train_scaled, y_train)
plt.show()

**Evaluation**

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
y_pred = knn.predict(X_test_scaled)

In [ ]:
y_pred

In [ ]:
print(classification_report(y_test, y_pred))

*Confusion Matrix (test)*


In [ ]:
print(confusion_matrix(y_test,y_pred))
knn_cm = confusion_matrix(y_test,y_pred)
display = ConfusionMatrixDisplay(confusion_matrix=knn_cm, display_labels = [False, True])
display.plot()
plt.show()

#**2. Random Forest Classification Algorithm** 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
Forest = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=5, random_state=42)
Forest.fit(X_train_scaled, y_train)

In [ ]:
Forest.score(X_test_scaled, y_test)

In [ ]:
Forest.score(X_train_scaled, y_train)

**ROC Curve**

In [ ]:
RocCurveDisplay.from_estimator(Forest, X_test_scaled, y_test)
plt.show()

In [ ]:
RocCurveDisplay.from_estimator(Forest, X_train_scaled, y_train)
plt.show()

**Evaluation**

In [ ]:
rf_y_pred = Forest.predict(X_test_scaled)

In [ ]:
rf_y_pred

In [ ]:
print(classification_report(y_test, rf_y_pred))

In [ ]:
print(confusion_matrix(y_test,rf_y_pred))

#**3. Neural Network Classification**

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape = (20,)))
model.add(keras.layers.Dense(200, activation = "relu"))
model.add(keras.layers.Dense(100, activation = "relu"))
model.add(keras.layers.Dense(100, activation = "relu"))
model.add(keras.layers.Dense(1, activation = "sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 5)


X_train_scaled_train, X_valid, y_train_train, y_valid = train_test_split(X_train_scaled, y_train, test_size=0.15)

model.fit(X_train_scaled_train, y_train_train, epochs=30, callbacks=[early_stopping_cb], validation_data=(X_valid, y_valid))

**Evaluation**

In [ ]:
model.evaluate(X_test_scaled, y_test)

NameError: ignored

In [ ]:
model.evaluate(X_train_scaled_train, y_train_train)

In [ ]:
nn_y_pred = model.predict(X_test_scaled)

**ROC Curve**

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, nn_y_pred.flatten())
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

In [ ]:
nn_y_pred

#**Deployment using Gradio**

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import joblib

In [ ]:

def csgo_classification(bomb_planted, ct_health , ct_armor , t_armor , ct_helmets , t_helmets , ct_defuse_kits , ct_players_alive , ct_weapon_ak47 , t_weapon_ak47 , ct_weapon_awp ,ct_weapon_m4a4, ct_weapon_sg553 , t_weapon_sg553, ct_weapon_usps , ct_grenade_hegrenade, ct_grenade_flashbang ,t_grenade_flashbang , ct_grenade_smokegrenade , ct_grenade_incendiarygrenade):
  my_list = [[bomb_planted, ct_health , ct_armor , t_armor , ct_helmets , t_helmets , ct_defuse_kits , ct_players_alive , ct_weapon_ak47 , t_weapon_ak47 ,ct_weapon_awp , ct_weapon_m4a4 , ct_weapon_sg553 , t_weapon_sg553, ct_weapon_usps , ct_grenade_hegrenade, ct_grenade_flashbang , t_grenade_flashbang,ct_grenade_smokegrenade , ct_grenade_incendiarygrenade]]

  y = model.predict(my_list)
  if y[0] == 1:
    return "T"
  else:
    return "CT"
inputs = [
    gr.inputs.Number(label="bomb_planted"),
    gr.inputs.Number(label="ct_health"),
    gr.inputs.Number(label="ct_armor"),
    gr.inputs.Number(label="t_armor"),
    gr.inputs.Number(label="ct_helmets"),
    gr.inputs.Number(label="t_helmets"),
    gr.inputs.Number(label="ct_defuse_kits"),
    gr.inputs.Number(label="ct_players_alive"),
    gr.inputs.Number(label="ct_weapon_ak47"),
    gr.inputs.Number(label="t_weapon_ak47"),
    gr.inputs.Number(label="t_weapon_awp"),
    gr.inputs.Number(label="ct_weapon_m4a4"),
    gr.inputs.Number(label="ct_weapon_sg553"),
    gr.inputs.Number(label="t_weapon_sg553"),
    gr.inputs.Number(label="ct_weapon_usps"),
    gr.inputs.Number(label="ct_grenade_hegrenade"),
    gr.inputs.Number(label="ct_grenade_flashbang"),
    gr.inputs.Number(label="t_grenade_flashbang"),
    gr.inputs.Number(label="ct_grenade_smokegrenade"),
    gr.inputs.Number(label="ct_grenade_incendiarygrenade")]

output = gr.outputs.Textbox(label="Prediction")

interface = gr.Interface(fn=csgo_classification, inputs=inputs, outputs=output, title="CS GO round winner classification")
interface.launch(share=True, debug=True)

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://19fa2673efe23bc738.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 58ms/step
